## Rc와 Weak 차이점 

### Rc 
- Rc는 참조 카운팅을 통해 런타임에서 소유권을 추적하고 객체가 소멸되는 시점을 결정하는 스마트 포인터입니다. 
- Rc는 다중 소유권(multiple ownership)을 지원하는 것으로도 알려져 있습니다.

### Weak 
- Weak 포인터는 강한 참조가 아닌 참조자이며, 객체의 소유권을 가지지 않습니다.
- 모든 Rc가 해제된 후에도 Weak 포인터는 여전히 유효하게 남아있을 수 있습니다. 
- 그러나 해당 객체를 직접 참조할 수 없으며, 대신 Weak 포인터를 통해 Rc를 참조할 수 있습니다. 
- Weak 포인터는 주로 순환 참조(circular references)를 방지하는 용도로 사용됩니다.

## 참조 카운팅

- Rc는 stack에 있는 데이터를 heap으로 옮겨주는 smart pointer입니다. 
- 이는 heap에 놓인 데이터를 immutable하게 대여하는 기능을 가진 다른 Rc smart pointer들을 복제할 수 있게 해줍니다.
- std::rc::Rc 모듈에서 제공하는 T 타입을 참조하는 불변 포인터로, 여러 개의 소유자가 공유하는 메모리를 사용할 수 있습니다.
- 마지막 smart pointer가 drop 될 때에만 heap에 있는 데이터가 할당 해제됩니다.

In [6]:
use std::rc::Rc;

In [13]:
fn main() {
    let heap_1 = Rc::new(100);
    let heap_2 = heap_1.clone();
    let heap_3 = heap_2.clone();
    println!("heap 1 ={}",heap_1);
    println!("heap 2 ={}",heap_2);
    println!("heap 2 ={}",heap_3);
}

In [12]:
main();

heap 1 =100
heap 2 =100


## 여러 개체 간에 값의 소유권을 공유합니다:

In [18]:
use std::rc::Rc;

#[derive(Debug)]
struct Foo {
    bar: Rc<String>,
}

#[derive(Debug)]
struct Baz {
    foo: Rc<Foo>,
}

fn main() {
    let foo = Foo {
        bar: Rc::new("Hello, world!".to_string()),
    };
    println!(" foo = {:?}", foo);
    
    let baz = Baz {
        foo: Rc::clone(&Rc::new(foo)),
    };
    
    //println!(" foo = {:?}", foo);
    println!(" baz = {:?}", baz);
}

In [19]:
main();

 foo = Foo { bar: "Hello, world!" }
 baz = Baz { foo: Foo { bar: "Hello, world!" } }


### 구조체를 Rc 처리하기

In [2]:
use std::rc::Rc;

struct Pie;

impl Pie {
    fn eat(&self) {
        println!("heap에 있으니 더 맛있습니다!")
    }
}

fn main() {
    let heap_pie = Rc::new(Pie);
    let heap_pie2 = heap_pie.clone();
    let heap_pie3 = heap_pie2.clone();

    heap_pie3.eat();
    heap_pie2.eat();
    heap_pie.eat();

    // 모든 참조 카운트 smart pointer가 여기서 drop 됩니다
    // heap 데이터인 Pie가 드디어 할당 해제됩니다
}


In [3]:
main();

heap에 있으니 더 맛있습니다!
heap에 있으니 더 맛있습니다!
heap에 있으니 더 맛있습니다!


### 구조체를 링크드리스트 처럼 처리하기 

In [5]:
use std::rc::Rc;

struct Node<T> {
    value: T,
    next: Option<Rc<Node<T>>>,
}

fn main() {
    let node1 = Rc::new(Node { value: 1, next: None });
    let node2 = Rc::new(Node { value: 2, next: Some(node1.clone()) });
    let node3 = Rc::new(Node { value: 3, next: Some(node2.clone()) });

    println!("Value of node1: {}", node1.value);
    println!("Value of node2: {}", node2.value);
    println!("Value of node3: {}", node3.value);

    match &node3.next {
        Some(node2_ref) => {
            println!("Value of node2 in node3: {}", node2_ref.value);
            match &node2_ref.next {
                Some(node1_ref) => {
                    println!("Value of node1 in node2: {}", node1_ref.value);
                },
                None => println!("End of the list."),
            }
        },
        None => println!("End of the list."),
    }
}

In [3]:
main();

Value of node1: 1
Value of node2: 2
Value of node3: 3
Value of node2 in node3: 2
Value of node1 in node2: 1


## Weak reference

- 약한 포인터는 객체가 삭제되는 것을 방지하지 않는 객체에 대한 참조입니다. 
- 즉, 객체에 대한 유일한 참조가 약 포인터인 경우 마지막 약 포인터가 삭제되면 객체가 삭제됩니다.

- 약한 포인터는 객체에 대한 소유권을 가지지 않고 객체의 수명을 추적하는 데 유용합니다. 
- 예를 들어, 자식 객체에만 관심이 있을 때 부모 객체를 추적하기 위해 약 포인터를 사용할 수 있습니다.

In [22]:
let weak_ptr:Weak<u32> = Weak::new();

In [23]:
let strong_ptr = weak_ptr.upgrade();

In [24]:
println!(" {:?}", strong_ptr);

 None


## 소유권을 갖지 않고도 가치의 수명을 추적할 수 있습니다:

In [20]:
use std::rc::Weak;

#[derive(Debug)]
struct Foo {
    bar: Weak<String>,
}

impl Drop for Foo {
    fn drop(&mut self) {
        if true {
            println!("The value has been dropped.");
        }
    }
}
    
fn main() {

    let foo = Foo {
        bar: Rc::downgrade(&Rc::new("Hello, world!".to_string())),
    };
    
    println!(" {:?}", foo);
}



In [21]:
main();

 Foo { bar: (Weak) }
The value has been dropped.


In [14]:
use std::rc::{Rc, Weak};
use std::cell::RefCell;

struct Node<T> {
    value: T,
    next: Option<Rc<RefCell<Node<T>>>>,
    prev: Option<Weak<RefCell<Node<T>>>>,
}

impl<T> Node<T> {
    fn new(value: T) -> Rc<RefCell<Self>> {
        let node = Rc::new(RefCell::new(Node {
            value: value,
            next: None,
            prev: None,
        }));
        let weak = Rc::downgrade(&node);
        node.borrow_mut().prev = Some(weak);
        node
    }
}

In [16]:
fn main() {
    let node1 = Node::new(1);
    let node2 = Node::new(2);
    let node3 = Node::new(3);

    node1.borrow_mut().next = Some(node2.clone());
    node2.borrow_mut().next = Some(node3.clone());
    node3.borrow_mut().next = Some(node1.clone());

    println!("Value of node1: {}", node1.borrow().value);
    println!("Value of node2: {}", node2.borrow().value);
    println!("Value of node3: {}", node3.borrow().value);

    match &node1.borrow().next {
        Some(node2_ref) => {
            println!("Value of node2 in node1: {}", node2_ref.borrow().value);
            match &node2_ref.borrow().next {
                Some(node3_ref) => {
                    println!("Value of node3 in node2: {}", node3_ref.borrow().value);
                    match &node3_ref.borrow().next {
                        Some(node1_ref) => {
                            println!("Value of node1 in node3: {}", node1_ref.borrow().value);
                        },
                        None => println!("End of the list."),
                    }
                },
                None => println!("End of the list."),
            }
        },
        None => println!("End of the list."),
    }
}


Error: no field `next` on type `Rc<RefCell<Node<{integer}>>>`